# EVERYTHING TRAINING

### CONFIG VARS

In [2]:
from utils import *
from ultralytics import YOLO

In [2]:
DATASET_PATH="../Datasets/Dataset"
PREPROCESSED_PATH="../Preprocessed_Datasets/Dataset"
AUGMENTED_PATH="../Augmented_Datasets/Dataset"
AUGMENTED_PATH_TRAIN_EX="../Augmented_Datasets/Dataset_train" # Data exclusively for training, not validating
YOLO_PATH="../YOLO_Datasets/Dataset"
YOLO_PATH_TRAIN_EX="../YOLO_Datasets/Dataset_train" # Data exclusively for training, not validating

In [4]:
if not os.path.exists(AUGMENTED_PATH):
    os.makedirs(AUGMENTED_PATH)
if not os.path.exists(AUGMENTED_PATH_TRAIN_EX):
    os.makedirs(AUGMENTED_PATH_TRAIN_EX)

### AUGMENTATION

For this model we will apply the following augmentation techniques:
- Hue transformations (-100º to +100º)
- Contrast inversion (To simulate dark and light modes)

In [10]:
create_slices(DATASET_PATH, AUGMENTED_PATH, 3, 3, 0.2, 0.2)

We apply resize now and not before because we want all images, including tiles, to keep the image size that the model will take

In [5]:
resize_dataset_images(DATASET_PATH, AUGMENTED_PATH, 640, 360)

In [6]:
hue_augmentation(AUGMENTED_PATH, AUGMENTED_PATH_TRAIN_EX, 0.15, 100)

In [7]:
contrast_inversion_augmentation(AUGMENTED_PATH, AUGMENTED_PATH_TRAIN_EX, 0.15)

In [8]:
# Add the augmented data to the train exclusively folder
for file in os.listdir(AUGMENTED_PATH):
    shutil.copy(os.path.join(AUGMENTED_PATH, file), AUGMENTED_PATH_TRAIN_EX)

### FORMAT CONVERSION

Up to now, we have treated with labelme format datasets, but we need to convert it to YOLOv8 format if we want to train
a model, which has the following format:

```
YOLOv8_Dataset/
├── data.yaml
├── train/
│   ├── images/
│   │   ├── img1.jpg
│   │   ├── img2.jpg
│   │   └── ...
│   ├── labels/
│   │   ├── img1.txt
│   │   ├── img2.txt
│   │   └── ...
├── valid/
│   ├── images/
│   │   ├── img1.jpg
│   │   ├── img2.jpg
│   │   └── ...
│   ├── labels/
│   │   ├── img1.txt
│   │   ├── img2.txt
│   │   └── ...
└── test/ (OPTIONAL)
    ├── images/
    │   ├── img1.jpg
    │   ├── img2.jpg
    │   └── ...
    └── labels/
        ├── img1.txt
        ├── img2.txt
        └── ...
```

The format of the data.yml file is:
```
path: <path_to_dataset_root_dit>
train: <path_to_train_images>
val: <path_to_validation_images>
test: <path_to_test_images> (OPTIONAL)

nc: <number_of_classes>
names: ['class1', 'class2', 'class3', ...]
```

The labels for Instance segmentation have the following format for each annotation:
```
<class-index> <x1> <y1> <x2> <y2> ... <xn> <yn>
```

In [9]:
labelme_to_yolo(AUGMENTED_PATH_TRAIN_EX, YOLO_PATH_TRAIN_EX, 0.7,[
                    "WebIcon",
                    "Icon",
                    "Switch",
                    "BtnSq",
                    "BtnPill",
                    "BtnCirc",
                    "CheckboxChecked",
                    "CheckboxUnchecked",
                    "RadiobtnSelected",
                    "RadiobtnUnselected",
                    "TextInput",
                    "Dropdown",
                    "Link",
                    "Text",
                    "TabActive",
                    "TabInactive",
                    "Sidebar",
                    "Navbar",
                    "Container",
                    "Image",
                    "BrowserURLInput",
                    "Header",
                    "Toolbar",
                    "BrowserToolbar",
                    "Scrollbar",
                    "Application",
                    "Taskbar",
                    "Dock",
                ], "seg")

### TRAIN

We will perform fine-tuning over the mobile-sam model using the hyperparameter tuning provided by Ultralytics to get the
best results we can. Since this is a non-standard dataset in terms of object features it is not clear what are the values
we should use.

We will also configure the training to not do any augmentation over the train set

In [ ]:
from ultralytics import YOLO
# Initialize the YOLO model
model = YOLO("yolov8s-seg")

In [11]:
import torch
# check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using {} device'.format(device))

Using cuda device


In [ ]:
# Tune hyperparameters on dataset for 30 epochs
model.train(data=f"{YOLO_PATH_TRAIN_EX}/data.yaml", workers=1, epochs=30, optimizer='AdamW', plots=False, save=True, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, fliplr=0.0)

In [3]:
model = YOLO("runs/segment/tune2/weights/best.pt")